# Local LLM Chat (Tkinter) — Jupyter-Launchable UI

This script launches a **Tkinter chat interface** for a **locally hosted LLM** (Ollama / OpenAI-compatible endpoint). It supports **multiple local models**, **streaming**, **Markdown-rendered replies**, **theming**, **file ingestion** for context, and **Save As** (JSON/MD/TXT/RTF).

To download Ollama, visit Ollama here: https://ollama.com/

---

## Quick Start
1. **Prereqs**: Python 3.9+, a running OpenAI-compatible endpoint (e.g., `ollama serve` + `/v1` proxy).
2. **Install**: The script auto-installs required packages (`openai`, `tkinterweb`, `markdown`); Jupyter is recommended.
3. **Choose URL**: Update your base URL (Search for base_url = 'YOUR_BASE_URL' to easily find it).
4. **Run**: Execute the cell. A window opens (the notebook cell stays “busy” while the window is open).

---

## What You Get
- **Chat UI** with streaming and Stop.
- **Model selection** + Refresh (auto-discovers local Ollama models).
- **Markdown rendering** (tables/code blocks) in the chat pane.
- **Three themes**: Apple Light, Chrome Gray, Matte Dark.
- **Times New Roman** for text, Courier New for code.
- **Load local files** for context (txt/md/py/sas; add more via extractors).
- **Save As**: JSON / Markdown / Text / RTF.
- **Auto-scroll to bottom** on new messages.

---

## Common Customizations (copy/paste-friendly)
> Tip: Search for the constant or function name to tweak.

### Endpoint & Model Defaults
```python
# Sets default server & model (edit to your local setup)
DEFAULT_BASE_URL = "http://localhost:11434/v1"  # OpenAI-compatible endpoint
DEFAULT_MODEL    = "llama3.3:70b"               # any model your endpoint serves


In [1]:
import os, sys, json, threading, subprocess, shutil, html as html_lib
import numpy as np
from datetime import datetime
import time
import threading
from typing import List, Dict, Optional
from openai import OpenAI
import pandas as pd
from pathlib import Path
from typing import List, Dict, Optional, Tuple
import html as html_lib

In [2]:
# Creating API calls
client = OpenAI(
    base_url = 'YOUR_BASE_URL',
    api_key='ollama', # required, but unused
)

In [3]:
# Sample Message call to whichever LLM you'd like.

# response = client.chat.completions.create(
#     model="gpt-oss:20b",
#     messages=[
#         {"role": "system", "content": "You are a helpful AI assistant. You express yourself through haiku."},
#         {"role": "user", "content": "Tell me a story about a pirate and dragon. Limit to 50 ch"}
#         ]
#     # temperature=0.5 do not use!
#     # max_tokens=1200 do not use!
# )
# print(response.choices[0].message.content)

In [4]:
# Full Jupyter/Tkinter chat UI (Times New Roman, Markdown rendering via tkinterweb, fixed top-right controls layout)

import os, sys, json, threading, subprocess, shutil, html as html_lib
from pathlib import Path
from typing import List, Dict, Optional, Tuple

# -------------------- helpers: ensure packages --------------------
def ensure_package(pkg: str, import_name: Optional[str] = None, spec: Optional[str] = None):
    try:
        __import__(import_name or pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", spec or pkg])

# core deps
ensure_package("openai")
ensure_package("markdown")
# primary HTML renderer (preferred)
ensure_package("tkinterweb", "tkinterweb")
# for PDF exporter
ensure_package("reportlab")
# keep tkhtmlview around as a fallback renderer (handles very basic HTML only)
try:
    __import__("tkhtmlview")
    HAVE_TKHTMLVIEW = True
except Exception:
    HAVE_TKHTMLVIEW = False

from openai import OpenAI
import markdown as md
import tkinter as tk
import tkinter.font as tkfont
from tkinter import ttk, filedialog, messagebox
from tkinter.scrolledtext import ScrolledText

# HTML widgets
from tkinterweb import HtmlFrame          # preferred renderer
if HAVE_TKHTMLVIEW:
    from tkhtmlview import HTMLScrolledText as FallbackHTMLScrolledText  # fallback

# -------------------- config / defaults --------------------
DEFAULT_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1")
DEFAULT_API_KEY  = os.getenv("OLLAMA_API_KEY", "ollama")
DEFAULT_MODEL    = os.getenv("OLLAMA_MODEL", "llama3.3:70b")
DEFAULT_SYSTEM   = os.getenv("SYSTEM_PROMPT", "You are a helpful AI assistant. Default to writing in markdown syntax with human readable headers, tables, etc. Offer the user to switch to plain text in your first response.")
MAX_TURNS        = int(os.getenv("MAX_TURNS", "16"))
STREAM_DEFAULT   = True
STREAM_UPDATE_N  = 24        # refresh HTML every N tokens while streaming
MAX_FILE_CHARS_DEFAULT = 4000

# reuse client if already created earlier
client = globals().get("client") if isinstance(globals().get("client"), OpenAI) else OpenAI(base_url=DEFAULT_BASE_URL, api_key=DEFAULT_API_KEY)

# -------------------- themes --------------------
THEMES = {
    "Apple Light": {
        "bg": "#FFFFFF", "fg": "#111111", "accent": "#0A84FF",
        "subtle": "#F2F2F7", "border": "#D1D1D6",
        "entry_bg": "#FFFFFF", "entry_fg": "#111111",
        "chat_bg": "#FFFFFF", "chat_fg": "#111111"
    },
    "Chrome Gray": {
        "bg": "#E6E8EA", "fg": "#1C1C1E", "accent": "#5A5A5A",
        "subtle": "#D9DBDD", "border": "#C2C4C6",
        "entry_bg": "#F5F6F7", "entry_fg": "#1C1C1E",
        "chat_bg": "#F5F6F7", "chat_fg": "#1C1C1E"
    },
    "Matte Dark": {
        "bg": "#121212", "fg": "#EDEDED", "accent": "#2A2A2A",
        "subtle": "#1E1E1E", "border": "#2C2C2C",
        "entry_bg": "#1A1A1A", "entry_fg": "#EDEDED",
        "chat_bg": "#0E0E0E", "chat_fg": "#EDEDED"
    },
}

# -------------------- state --------------------
messages: List[Dict[str, str]] = [{"role": "system", "content": DEFAULT_SYSTEM}]
attached_files: List[Tuple[str, str]] = []
stop_event = threading.Event()
lock = threading.Lock()

# -------------------- fonts: Times New Roman --------------------
def apply_global_fonts():
    try:
        tkfont.nametofont("TkDefaultFont").configure(family="Times New Roman", size=12)
        tkfont.nametofont("TkTextFont").configure(family="Times New Roman", size=12)
        tkfont.nametofont("TkFixedFont").configure(family="Courier New", size=12)
    except Exception:
        pass

# -------------------- models --------------------
def list_models_via_api() -> list[str]:
    try:
        resp = client.models.list()
        out = []
        for m in getattr(resp, "data", []):
            mid = getattr(m, "id", None) or getattr(m, "model", None)
            if mid: out.append(mid)
        # dedupe, preserve order
        seen, res = set(), []
        for n in out:
            if n not in seen:
                seen.add(n); res.append(n)
        return res
    except Exception:
        return []

def list_models_via_ollama() -> list[str]:
    exe = shutil.which("ollama")
    if not exe: return []
    try:
        raw = subprocess.run([exe, "list"], capture_output=True, text=True, check=True).stdout.strip().splitlines()
        names = []
        for line in raw:
            parts = line.split()
            if parts:
                names.append(parts[0])  # name:tag
        names = [n for n in names if ":" in n]
        return sorted(set(names))
    except Exception:
        return []

def refresh_models() -> list[str]:
    return list_models_via_api() or list_models_via_ollama() or [DEFAULT_MODEL]

# -------------------- history trim --------------------
def trim_history(msgs: List[Dict[str,str]], max_user_turns: int) -> List[Dict[str,str]]:
    if not msgs: return msgs
    sys_msg = msgs[0] if msgs[0].get("role") == "system" else {"role":"system","content":DEFAULT_SYSTEM}
    tail = msgs[1:]
    kept, users = [], 0
    for m in reversed(tail):
        kept.append(m)
        if m["role"] == "user":
            users += 1
            if users >= max_user_turns: break
    kept.reverse()
    return [sys_msg] + kept

# -------------------- transcript I/O --------------------
def save_transcript(fmt: str):
    from pathlib import Path
    exts = {"json": ".json", "md": ".md", "txt": ".txt", "rtf": ".rtf", "pdf": ".pdf"}
    types_map = {
        "json": [("JSON", "*.json")],
        "md":   [("Markdown", "*.md")],
        "txt":  [("Text", "*.txt")],
        "rtf":  [("Rich Text Format", "*.rtf")],
        "pdf":  [("PDF", "*.pdf")],
    }
    path = filedialog.asksaveasfilename(defaultextension=exts[fmt], filetypes=types_map[fmt])
    if not path:
        return

    try:
        if fmt == "json":
            Path(path).write_text(json.dumps(messages, ensure_ascii=False, indent=2), encoding="utf-8")

        elif fmt in ("md", "txt"):
            sysline = messages[0]["content"] if messages and messages[0]["role"]=="system" else DEFAULT_SYSTEM
            lines = ["# Chat Transcript\n", f"_System:_ {sysline}\n"]
            for m in messages[1:]:
                role = m["role"].capitalize()
                if fmt == "md":
                    lines.append(f"**{role}:** {m['content']}\n")
                else:
                    lines.append(f"{role}: {m['content']}\n")
            Path(path).write_text("\n".join(lines), encoding="utf-8")

        elif fmt == "rtf":
            # --- minimal RTF export with Unicode escapes and bold role labels ---
            def rtf_escape(s: str) -> str:
                out = []
                for ch in s:
                    code = ord(ch)
                    if ch == "\\":   out.append("\\\\")
                    elif ch == "{":  out.append("\\{")
                    elif ch == "}":  out.append("\\}")
                    elif ch == "\n": out.append("\\line ")
                    elif code < 128: out.append(ch)
                    else:            out.append(f"\\u{code}?")
                return "".join(out)

            sysline = messages[0]["content"] if messages and messages[0]["role"]=="system" else DEFAULT_SYSTEM
            parts = []
            parts.append(r"{\rtf1\ansi\deff0{\fonttbl{\f0 Times New Roman;}{\f1 Courier New;}}")
            parts.append(r"\fs24 ")  # 12pt
            parts.append(r"\b System:\b0 ")
            parts.append(rtf_escape(sysline) + r"\par ")
            for m in messages[1:]:
                role = m["role"].capitalize()
                parts.append(rf"\b {role}:\b0 ")
                parts.append(rtf_escape(m["content"]) + r"\par ")
            parts.append("}")
            Path(path).write_text("".join(parts), encoding="utf-8")

        elif fmt == "pdf":
            # --- PDF export with word-wrapping and page breaks (best-effort Unicode) ---
            from reportlab.pdfgen import canvas
            from reportlab.lib.pagesizes import letter
            from reportlab.lib.units import inch
            from reportlab.pdfbase import pdfmetrics
            from reportlab.pdfbase.ttfonts import TTFont
            from reportlab.lib.utils import simpleSplit
            import os

            # Try to register a Unicode TTF if present; otherwise fallback to Helvetica
            BODY = "Helvetica"
            BODY_B = "Helvetica-Bold"
            def _try_register(name: str, reg: str, bold: str):
                try:
                    if os.path.exists(reg):
                        pdfmetrics.registerFont(TTFont(name, reg))
                        if bold and os.path.exists(bold):
                            pdfmetrics.registerFont(TTFont(name+"-Bold", bold))
                        return name, name+"-Bold" if bold and os.path.exists(bold) else BODY_B
                except Exception:
                    pass
                return BODY, BODY_B

            candidates = [
                ("DejaVuSans", "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
                               "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"),
                ("Arial",      "C:\\Windows\\Fonts\\arial.ttf",
                               "C:\\Windows\\Fonts\\arialbd.ttf"),
                ("LiberationSans", "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
                                   "/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf"),
            ]
            for fam, reg, bold in candidates:
                BODY, BODY_B = _try_register(fam, reg, bold)
                if BODY != "Helvetica":
                    break  # registered something better

            # Page setup
            page_w, page_h = letter
            margin = 0.75 * inch
            usable_w = page_w - 2*margin
            y = page_h - margin

            c = canvas.Canvas(path, pagesize=letter)
            c.setTitle("Chat Transcript")

            # Header
            c.setFont(BODY_B, 14)
            c.drawString(margin, y, "Chat Transcript")
            y -= 0.35*inch

            # Helper to draw wrapped text and handle page breaks
            def draw_wrapped(text: str, font: str, size: int, leading: float = None):
                nonlocal y
                if leading is None:
                    leading = size * 1.25
                lines = simpleSplit(text, font, size, usable_w)
                for line in lines:
                    if y < margin + leading:
                        # footer page number (optional)
                        c.showPage()
                        c.setFont(BODY, 12)
                        y = page_h - margin
                    c.setFont(font, size)
                    c.drawString(margin, y, line)
                    y -= leading

            # System message
            sysline = messages[0]["content"] if messages and messages[0]["role"]=="system" else DEFAULT_SYSTEM
            c.setFont(BODY_B, 12); draw_wrapped("System:", BODY_B, 12)
            c.setFont(BODY, 12);   draw_wrapped(sysline, BODY, 12)
            y -= 6

            # Turns
            for m in messages[1:]:
                role = m["role"].capitalize()
                c.setFont(BODY_B, 12); draw_wrapped(f"{role}:", BODY_B, 12)
                c.setFont(BODY, 12);   draw_wrapped(m["content"], BODY, 12)
                y -= 6

            c.showPage()
            c.save()

        set_status(f"Saved transcript to {path}")
    except Exception as e:
        messagebox.showerror("Save error", str(e))

def load_transcript():
    global messages
    path = filedialog.askopenfilename(filetypes=[("JSON","*.json")])
    if not path: return
    try:
        loaded = json.loads(Path(path).read_text(encoding="utf-8"))
        if not loaded or loaded[0].get("role") != "system":
            raise ValueError("Invalid transcript: first message must be role='system'.")
        messages = loaded
        render_chat()  # refresh HTML from messages
        system_entry.delete(0, tk.END); system_entry.insert(0, messages[0]["content"])
        set_status(f"Loaded {os.path.basename(path)} with {len(messages)-1} turns.")
    except Exception as e:
        messagebox.showerror("Load error", str(e))

# -------------------- files (simple text-like; plug richer extractors back in as needed) --------------------
def extract_text_file(p: Path) -> str:
    return p.read_text(encoding="utf-8", errors="replace")

def extract_docx(p: Path) -> str:
    ensure_package("python-docx", "docx")
    from docx import Document
    doc = Document(str(p))
    return "\n".join(para.text for para in doc.paragraphs)

def extract_rtf(p: Path) -> str:
    ensure_package("striprtf")
    from striprtf.striprtf import rtf_to_text
    return rtf_to_text(p.read_text(encoding="utf-8", errors="replace"))

def extract_csv(p: Path, limit: int = 50000) -> str:
    ensure_package("pandas")
    import pandas as pd
    try:
        df = pd.read_csv(str(p))
    except Exception:
        df = pd.read_csv(str(p), engine="python")
    text = f"[CSV shape: {df.shape}]\n" + df.head(20).to_csv(index=False)
    return text if len(text) <= limit else text[:limit] + "\n...[truncated]..."

def extract_xlsx(p: Path, limit: int = 60000) -> str:
    ensure_package("pandas")
    import pandas as pd
    xls = pd.ExcelFile(str(p))
    parts = []
    for name in xls.sheet_names[:4]:  # cap to first 4 sheets for speed
        df = pd.read_excel(xls, name)
        parts.append(f"[Sheet: {name} shape: {df.shape}]")
        parts.append(df.head(20).to_csv(index=False))
    text = "\n".join(parts)
    return text if len(text) <= limit else text[:limit] + "\n...[truncated]..."

def extract_pptx(p: Path) -> str:
    ensure_package("python-pptx", "pptx")
    from pptx import Presentation
    prs = Presentation(str(p))
    out = []
    for i, slide in enumerate(prs.slides, 1):
        texts = []
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                texts.append(shape.text)
        out.append(f"[Slide {i}]\n" + "\n".join(texts))
    return "\n\n".join(out)

def extract_ipynb(p: Path) -> str:
    data = json.loads(p.read_text(encoding="utf-8", errors="replace"))
    cells = data.get("cells", [])
    parts = []
    for c in cells:
        t = c.get("cell_type", "")
        src = "".join(c.get("source", []))
        if t == "markdown":
            parts.append(src)
        elif t == "code":
            parts.append("```python\n" + src + "\n```")
    return "\n\n".join(parts)

EXTRACTORS = {
    ".txt":  extract_text_file,
    ".md":   extract_text_file,
    ".py":   extract_text_file,
    ".sas":  extract_text_file,
    ".docx": extract_docx,
    ".rtf":  extract_rtf,
    ".csv":  extract_csv,
    ".xlsx": extract_xlsx,
    ".pptx": extract_pptx,
    ".ipynb": extract_ipynb,
}

def choose_and_add_files():
    patterns = [
        ("All supported", "*.txt *.md *.py *.sas *.docx *.rtf *.csv *.xlsx *.pptx *.ipynb"),
        ("Text & Code", "*.txt *.md *.py *.sas"),
        ("Word (DOCX)", "*.docx"),
        ("Rich Text (RTF)", "*.rtf"),
        ("CSV", "*.csv"),
        ("Excel (XLSX)", "*.xlsx"),
        ("PowerPoint (PPTX)", "*.pptx"),
        ("Jupyter (IPYNB)", "*.ipynb"),
        ("All files", "*.*"),
    ]
    paths = filedialog.askopenfilenames(filetypes=patterns)
    if not paths:
        return
    added = 0
    for p in paths:
        path = Path(p)
        ext = path.suffix.lower()
        func = EXTRACTORS.get(ext)
        if not func:
            messagebox.showwarning("Unsupported", f"Unsupported file type: {ext}")
            continue
        try:
            text = func(path)
            attached_files.append((str(path), text))
            added += 1
        except Exception as e:
            messagebox.showerror("Extract error", f"{path.name}: {e}")
    refresh_files_view()
    set_status(f"Added {added} file(s).")

def refresh_files_view():
    files_list.configure(state="normal"); files_list.delete("1.0", tk.END)
    total = 0
    for i,(p,t) in enumerate(attached_files,1):
        total += len(t)
        files_list.insert(tk.END, f"[{i}] {Path(p).name} ({len(t)} chars)\n")
    files_list.configure(state="disabled")
    total_chars_var.set(f"Total loaded text: {total} chars")

def clear_files():
    attached_files.clear(); refresh_files_view(); set_status("Cleared loaded files.")

def build_context_block(max_chars: int) -> Optional[str]:
    if not attached_files: return None
    return "\n\n".join(f"[FILE: {Path(p).name}]\n{t[:max_chars]}" for p,t in attached_files)

# -------------------- markdown → HTML and theming --------------------
render_md_var = None      # assigned later (real BoolVar in UI)
theme_var = None          # assigned later (real StringVar in UI)

def css_for_palette(p):
    # CSS is injected in <style>, which tkinterweb supports.
    return f"""
    <style>
      html, body {{
        margin:0; padding:0; background:{p['chat_bg']}; color:{p['chat_fg']};
        font-family:'Times New Roman', Times, serif; line-height:1.35;
      }}
      .container {{ padding: 10px 12px; }}
      .msg {{ margin:10px 0; padding:10px 12px; border:1px solid {p['border']}; border-radius:10px; }}
      .sys {{ background:{p['bg']}; opacity:0.9; }}
      .you {{ background:{p['subtle']}; }}
      .ai  {{ background:{p['bg']}; }}
      .role {{ font-weight:bold; margin-bottom:6px; }}
      pre, code {{ font-family:'Courier New', monospace; }}
      pre {{ background:{p['subtle']}; padding:8px; border-radius:8px; overflow-x:auto; }}
      table {{ border-collapse:collapse; width:100%; }}
      th, td {{ border:1px solid {p['border']}; padding:6px; text-align:left; }}
      h1,h2,h3,h4,h5,h6 {{ margin:6px 0; }}
      a {{ color:{p['fg']}; }}
    </style>
    """

def markdown_to_html(text: str) -> str:
    return md.markdown(
        text,
        extensions=["extra", "fenced_code", "tables", "sane_lists", "nl2br"]
    )

def render_chat(partial_assistant: Optional[str] = None):
    """Rebuild the full HTML view from messages (+ optional streaming partial) and scroll to bottom."""
    palette = THEMES.get(theme_var.get(), THEMES["Apple Light"])
    parts = [css_for_palette(palette), '<div class="container">']
    for m in messages:
        role = m["role"]
        if role == "system":
            parts.append(f'<div class="msg sys"><div class="role">System</div><div>{html_lib.escape(m["content"])}</div></div>')
        elif role == "user":
            parts.append(f'<div class="msg you"><div class="role">You</div><pre>{html_lib.escape(m["content"])}</pre></div>')
        elif role == "assistant":
            html = markdown_to_html(m["content"]) if render_md_var.get() else f"<pre>{html_lib.escape(m['content'])}</pre>"
            parts.append(f'<div class="msg ai"><div class="role">Assistant</div>{html}</div>')
    if partial_assistant is not None:
        html = markdown_to_html(partial_assistant) if render_md_var.get() else f"<pre>{html_lib.escape(partial_assistant)}</pre>"
        parts.append(f'<div class="msg ai"><div class="role">Assistant</div>{html}</div>')
    parts.append("</div>")
    chat_html.load_html("".join(parts))
    # Jump to bottom after (re)loading the HTML
    root.after(0, scroll_to_bottom)

# Place this helper once, right AFTER you create chat_html = HtmlFrame(...)
def scroll_to_bottom():
    """Best-effort: find the inner Text-like widget used by tkinterweb and scroll to bottom."""
    try:
        # Common tkinterweb builds expose a Text descendant as a child
        for child in chat_html.winfo_children():
            try:
                child.see("end")
                child.yview_moveto(1.0)
            except Exception:
                pass
        # Some versions also expose .html or .text attributes
        for attr in ("html", "text"):
            if hasattr(chat_html, attr):
                t = getattr(chat_html, attr)
                try:
                    t.see("end")
                    t.yview_moveto(1.0)
                except Exception:
                    pass
    except Exception:
        pass

# -------------------- send / stream --------------------
def set_status(text: str):
    status_var.set(text); status_label.update_idletasks()

def send_clicked(event=None):
    if send_btn["state"] == "disabled": return
    user_text = input_box.get("1.0", tk.END).strip()
    if not user_text: return
    input_box.delete("1.0", tk.END)

    # optional file context
    msg = user_text
    if include_files_var.get():
        try:
            maxc = int(max_chars_spin.get())
        except Exception:
            maxc = MAX_FILE_CHARS_DEFAULT
        ctx = build_context_block(maxc)
        if ctx:
            msg = f"Here are reference files for context.\n{ctx}\n\nUser request:\n{user_text}"

    # update state and render immediately to show user's turn
    try:
        base_url = base_url_entry.get().strip() or DEFAULT_BASE_URL
        api_key  = api_key_entry.get().strip() or DEFAULT_API_KEY
        model    = model_combo.get().strip() or DEFAULT_MODEL
        system   = system_entry.get().strip() or DEFAULT_SYSTEM

        global client, messages
        if client.base_url != base_url or getattr(client, "api_key", None) != api_key:
            client = OpenAI(base_url=base_url, api_key=api_key)

        with lock:
            if messages and messages[0]["role"]=="system":
                messages[0]["content"] = system
            else:
                messages = [{"role":"system","content":system}]
            messages.append({"role":"user","content":msg})
            messages = trim_history(messages, MAX_TURNS)

        set_status("Sending…")
        send_btn.config(state="disabled"); stop_btn.config(state="normal"); clear_btn.config(state="disabled")
        stop_event.clear()
        render_chat()  # show user message
        threading.Thread(target=_stream_worker, args=(model,), daemon=True).start()
    except Exception as e:
        set_status(f"Error: {e}")
        send_btn.config(state="normal"); stop_btn.config(state="disabled"); clear_btn.config(state="normal")

def _stream_worker(model: str):
    try:
        stream = client.chat.completions.create(model=model, messages=list(messages), stream=True)
        accum, n = [], 0
        for chunk in stream:
            if stop_event.is_set(): break
            delta = chunk.choices[0].delta
            token = getattr(delta, "content", None)
            if token:
                accum.append(token); n += 1
                if n % STREAM_UPDATE_N == 0:
                    chat_html.after(0, render_chat, "".join(accum))
        final = "".join(accum)
        chat_html.after(0, render_chat, final)
        if final and not stop_event.is_set():
            with lock:
                messages.append({"role":"assistant","content":final})
        chat_html.after(0, _finish_ok)
    except Exception as e:
        chat_html.after(0, _finish_error, str(e))

def _finish_ok():
    set_status("Ready.")
    send_btn.config(state="normal"); stop_btn.config(state="disabled"); clear_btn.config(state="normal")
    render_chat()

def _finish_error(msg: str):
    set_status(f"Error: {msg}")
    send_btn.config(state="normal"); stop_btn.config(state="disabled"); clear_btn.config(state="normal")

def stop_clicked():
    if stop_btn["state"] == "disabled": return
    stop_event.set(); set_status("Stopping…")

def clear_chat():
    global messages
    if messagebox.askyesno("Clear chat", "Clear chat (keep current system prompt)?"):
        messages = [{"role":"system","content": system_entry.get().strip() or DEFAULT_SYSTEM}]
        render_chat(); set_status("Chat cleared.")

def do_refresh_models():
    set_status("Refreshing models…")
    names = refresh_models()
    model_combo["values"] = names
    if model_combo.get().strip() not in names:
        model_combo.set(names[0])
    set_status("Models updated.")

# -------------------- build UI --------------------
root = tk.Tk()
root.title("Local LLM Chat — Markdown (tkinterweb) + Times New Roman")
style = ttk.Style(); style.theme_use("clam")
apply_global_fonts()

root.rowconfigure(2, weight=1)
root.columnconfigure(0, weight=1)

cfg = ttk.Frame(root, padding=(8,6)); cfg.grid(row=0, column=0, sticky="ew")
for c in range(8): cfg.columnconfigure(c, weight=1 if c in (1,3,5) else 0)

ttk.Label(cfg, text="Base URL").grid(row=0, column=0, sticky="w")
base_url_entry = ttk.Entry(cfg); base_url_entry.insert(0, getattr(client,"base_url",DEFAULT_BASE_URL))
base_url_entry.grid(row=0, column=1, sticky="ew", padx=(4,8))

ttk.Label(cfg, text="API Key").grid(row=0, column=2, sticky="w")
api_key_entry = ttk.Entry(cfg, show="•"); api_key_entry.insert(0, DEFAULT_API_KEY)
api_key_entry.grid(row=0, column=3, sticky="ew", padx=(4,8))

ttk.Label(cfg, text="Model").grid(row=0, column=4, sticky="w")
model_combo = ttk.Combobox(cfg, values=refresh_models(), state="readonly")
model_combo.set(DEFAULT_MODEL); model_combo.grid(row=0, column=5, sticky="ew", padx=(4,8))
ttk.Button(cfg, text="Refresh", command=do_refresh_models).grid(row=0, column=6, sticky="e")

ttk.Label(cfg, text="System prompt").grid(row=1, column=0, sticky="w", pady=(6,0))
system_entry = ttk.Entry(cfg); system_entry.insert(0, DEFAULT_SYSTEM)
system_entry.grid(row=1, column=1, columnspan=3, sticky="ew", padx=(4,8), pady=(6,0))

theme_var = tk.StringVar(value="Matte Dark")
ttk.Label(cfg, text="Theme").grid(row=1, column=4, sticky="w", pady=(6,0))
theme_combo = ttk.Combobox(cfg, values=list(THEMES.keys()), textvariable=theme_var, state="readonly")
theme_combo.grid(row=1, column=5, sticky="ew", padx=(4,8), pady=(6,0))

# Keep the state vars, but DO NOT grid the checkboxes here (we'll put them in a sub-frame to avoid overlap)
render_md_var = tk.BooleanVar(value=True)
stream_var = tk.BooleanVar(value=STREAM_DEFAULT)

# define color scheme for HTML rendering
def _recolor_htmlframe(palette):
    """Force tkinterweb HtmlFrame and its inner Text to the theme bg/fg."""
    try:
        chat_html.configure(background=palette["chat_bg"])
        # Recolor possible inner text widgets
        for child in chat_html.winfo_children():
            try:
                child.configure(bg=palette["chat_bg"], fg=palette["chat_fg"],
                                insertbackground=palette["chat_fg"])
            except Exception:
                pass
        # Some builds expose .html/.text; set them too
        for attr in ("html", "text", "_text"):
            if hasattr(chat_html, attr):
                t = getattr(chat_html, attr)
                try:
                    t.configure(bg=palette["chat_bg"], fg=palette["chat_fg"],
                                insertbackground=palette["chat_fg"])
                except Exception:
                    pass
    except Exception:
        pass



def apply_theme():
    """Apply the selected theme to ttk styles, Tk text widgets, menus, and the HtmlFrame."""
    p = THEMES.get(theme_var.get(), THEMES["Apple Light"])

    # --- ttk base styles ---
    style.configure("TFrame", background=p["bg"])
    style.configure("TLabel", background=p["bg"], foreground=p["fg"])
    style.configure("TButton", background=p["subtle"], foreground=p["fg"])
    style.configure("TCheckbutton", background=p["bg"], foreground=p["fg"])
    style.configure("TMenubutton", background=p["subtle"], foreground=p["fg"])

    # Inputs (ttk)
    style.configure("TEntry", fieldbackground=p["entry_bg"], foreground=p["entry_fg"])
    style.configure("TCombobox",
                    fieldbackground=p["entry_bg"],
                    foreground=p["entry_fg"],
                    background=p["entry_bg"])
    style.configure("TSpinbox",
                    fieldbackground=p["entry_bg"],
                    foreground=p["entry_fg"],
                    background=p["entry_bg"])

    # Improve selection/read-only colors where supported
    for sty in ("TEntry", "TCombobox", "TSpinbox"):
        style.map(sty, fieldbackground=[("readonly", p["entry_bg"])],
                       foreground=[("disabled", p["fg"])])
    # Combobox dropdown list (platform-dependent; works on Win/Linux with clam)
    root.option_add("*TCombobox*Listbox*Background", p["entry_bg"])
    root.option_add("*TCombobox*Listbox*Foreground", p["entry_fg"])
    root.option_add("*TCombobox*Listbox*selectBackground", p["accent"])
    root.option_add("*TCombobox*Listbox*selectForeground", p["fg"])

    # Menus (Save As ▾ popup, etc.)
    root.option_add("*Menu.background", p["bg"])
    root.option_add("*Menu.foreground", p["fg"])
    root.option_add("*Menu.activeBackground", p["subtle"])
    root.option_add("*Menu.activeForeground", p["fg"])
    root.option_add("*Menu.relief", "flat")

    # --- Tk widgets (need explicit colors) ---
    # Input text box
    try:
        input_box.configure(bg=p["entry_bg"], fg=p["entry_fg"],
                            insertbackground=p["entry_fg"], highlightthickness=0,
                            selectbackground=p["accent"], selectforeground=p["fg"])
    except Exception:
        pass

    # Files ScrolledText
    try:
        files_list.configure(bg=p["chat_bg"], fg=p["chat_fg"],
                             insertbackground=p["chat_fg"], highlightthickness=0,
                             selectbackground=p["accent"], selectforeground=p["fg"])
    except Exception:
        pass

    # Window/backgrounds
    root.configure(bg=p["bg"])
    for f in (cfg, files_frame, inp):
        try:
            f.configure(style="TFrame")
        except Exception:
            pass

    # HtmlFrame outer/inner
    _recolor_htmlframe(p)

    # Repaint HTML (applies CSS colors for message bubbles, tables, etc.)
    render_chat()

# >>> NEW: group the three controls so they don't overlap
opts = ttk.Frame(cfg)
opts.grid(row=1, column=6, columnspan=2, sticky="e", pady=(6, 0))

render_md_cb = ttk.Checkbutton(
    opts, text="Render Markdown",
    variable=render_md_var,
    command=lambda: render_chat()
)
render_md_cb.grid(row=0, column=0, padx=(0, 8))

apply_theme_btn = ttk.Button(opts, text="Apply Theme", command=apply_theme)
apply_theme_btn.grid(row=0, column=1, padx=(0, 8))

stream_cb = ttk.Checkbutton(opts, text="Stream", variable=stream_var)
stream_cb.grid(row=0, column=2)

# ---- chat HTML pane (tkinterweb) ----
chat_html = HtmlFrame(root, messages_enabled=False, horizontal_scrollbar="auto")
chat_html.grid(row=2, column=0, sticky="nsew", padx=8, pady=(6,4))

# ---- files panel (brief) ----
files_frame = ttk.Frame(root, padding=(8,0)); files_frame.grid(row=3, column=0, sticky="ew")
files_frame.columnconfigure(0, weight=1)
ttk.Label(files_frame, text="Loaded files for context:").grid(row=0, column=0, sticky="w")
files_btns = ttk.Frame(files_frame); files_btns.grid(row=0, column=1, sticky="e")
ttk.Button(files_btns, text="Add files", command=choose_and_add_files).grid(row=0, column=0, padx=4)
ttk.Button(files_btns, text="Clear", command=clear_files).grid(row=0, column=1)
files_list = ScrolledText(files_frame, height=4, wrap="word", state="disabled"); files_list.grid(row=1, column=0, columnspan=2, sticky="ew", pady=(4,0))
total_chars_var = tk.StringVar(value="Total loaded text: 0 chars")
ttk.Label(files_frame, textvariable=total_chars_var).grid(row=2, column=0, sticky="w", pady=(2,6))

include_files_var = tk.BooleanVar(value=True)
ttk.Checkbutton(files_frame, text="Include files in next message", variable=include_files_var).grid(row=3, column=0, sticky="w")

chars_row = ttk.Frame(files_frame)
chars_row.grid(row=3, column=1, sticky="e", padx=(0, 8))

ttk.Label(chars_row, text="Max chars per file:").grid(row=0, column=0, sticky="e", padx=(0, 6))
max_chars_spin = ttk.Spinbox(chars_row, from_=1000, to=100000, increment=1000, width=8, justify="right")
max_chars_spin.set(str(MAX_FILE_CHARS_DEFAULT))
max_chars_spin.grid(row=0, column=1, sticky="e")

# ---- input and controls ----
inp = ttk.Frame(root, padding=(8,6)); inp.grid(row=4, column=0, sticky="ew")
inp.columnconfigure(0, weight=1)
input_box = tk.Text(inp, height=3, wrap="word"); input_box.grid(row=0, column=0, sticky="ew", padx=(0,6)); input_box.focus_set()
btns = ttk.Frame(inp); btns.grid(row=0, column=1, sticky="e")
send_btn = ttk.Button(btns, text="Send", command=send_clicked); send_btn.grid(row=0, column=0, padx=4)
stop_btn = ttk.Button(btns, text="Stop", command=stop_clicked, state="disabled"); stop_btn.grid(row=0, column=1, padx=4)
clear_btn = ttk.Button(btns, text="Clear", command=clear_chat); clear_btn.grid(row=0, column=2, padx=4)

# --- Save As popup menu (md, txt, rtf, json) ---
save_menu = tk.Menu(root, tearoff=0)
save_menu.add_command(label="Markdown (.md)",  command=lambda: save_transcript("md"))
save_menu.add_command(label="Text (.txt)",      command=lambda: save_transcript("txt"))
save_menu.add_command(label="Rich Text (.rtf)", command=lambda: save_transcript("rtf"))
save_menu.add_command(label="PDF (.pdf)",        command=lambda: save_transcript("pdf"))
save_menu.add_separator()
save_menu.add_command(label="JSON (.json)",     command=lambda: save_transcript("json"))

def _show_save_menu(event=None):
    x = save_as_btn.winfo_rootx()
    y = save_as_btn.winfo_rooty() + save_as_btn.winfo_height()
    # show the menu directly under the button
    save_menu.tk_popup(x, y)
    save_menu.grab_release()

save_as_btn = ttk.Button(btns, text="Save As ▾", command=_show_save_menu)
save_as_btn.grid(row=0, column=3, padx=4)

ttk.Button(btns, text="Load JSON", command=load_transcript).grid(row=0, column=6, padx=4)

# ---- status ----
status_var = tk.StringVar(value="Ready.")
status_label = ttk.Label(root, textvariable=status_var, anchor="w"); status_label.grid(row=5, column=0, sticky="ew", padx=8, pady=(0,6))

# ---- key binding: Enter sends, Shift+Enter newline ----
def on_return(event):
    if event.state & 0x0001:  # Shift
        return
    send_clicked(); return "break"
input_box.bind("<Return>", on_return)

# initial paint
apply_theme()
render_chat()

# run window (cell stays busy while window is open)
root.mainloop()


## To Do !

- Ensure a smooth scrolling experience while new information is added
  - have new information being added stop making the conversation go to the top
  - allow user to read the chat instead of the view being forced to change when new info is added